In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Imports

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

Pandas Settings:

In [26]:
pd.set_option("max_colwidth", 150)
pd.set_option("display.max_columns", 60)
pd.set_option('max_rows', None)
# pd.set_option("max_colwidth", 250)

In [15]:
df_train = pd.read_csv(r'./train.csv')
df_test = pd.read_csv(r'./test.csv')
df_sub = pd.read_csv(r'./gender_submission.csv')

display(df_train)
display(df_test)
# display(df_sub)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Okay heres the order for the MVP training set:
1. Drop non-MVP columns
1. Drop NaN values
1. OHC Sex column
1. Split X and Y
1. Fit
1. Score

1. Drop non-MVP columns from both train and test sets

In [34]:
train_copy = df_train.copy()
train_mvp = train_copy.drop(['PassengerId','Name','Ticket','Cabin',"Embarked"], axis=1)
display(train_mvp.head(5))

test_copy = df_test.copy()
test_mvp = test_copy.drop(['PassengerId','Name','Ticket','Cabin',"Embarked"], axis=1)
display(test_mvp.head(5))

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875


Checking for NaN values in Sex column

In [53]:
print("df_train:\n", df_train.isnull().sum())
print()
print(train_mvp.isnull().sum())

# df.isnull().sum()

df_train:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
dtype: int64


3. OHC Sex Column

In [101]:
train_mvp_c = train_mvp.copy()
train_mvp_c = pd.get_dummies(train_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
display(train_mvp_c.head(25))

test_mvp_c = test_mvp.copy()
test_mvp_c = pd.get_dummies(test_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
# display(test_mvp_c.head(25))

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1
5,0,3,NaN,0,0,8.4583,1
6,0,1,54.0,0,0,51.8625,1
7,0,3,2.0,3,1,21.0750,1
8,1,3,27.0,0,2,11.1333,0
9,1,2,14.0,1,0,30.0708,0


2. Imputting NaN values

In [107]:
imputer = IterativeImputer()
# imputer = KNNImputer(n_neighbors=8)

train_mvp_c = imputer.fit_transform(train_mvp_c)
train_mvp_c = pd.DataFrame(train_mvp_c)
display(train_mvp_c.head(20))

# train_mvp_c = train_mvp_c.reindex_like(train_mvp)
# columns = ['Survived', Pclass]


,0,1,2,3,4,5,6
0,0.0,3.0,22.000000,1.0,0.0,7.2500,1.0
1,1.0,1.0,38.000000,1.0,0.0,71.2833,0.0
2,1.0,3.0,26.000000,0.0,0.0,7.9250,0.0
3,1.0,1.0,35.000000,1.0,0.0,53.1000,0.0
4,0.0,3.0,35.000000,0.0,0.0,8.0500,1.0
5,0.0,3.0,29.087022,0.0,0.0,8.4583,1.0
6,0.0,1.0,54.000000,0.0,0.0,51.8625,1.0
7,0.0,3.0,2.000000,3.0,1.0,21.0750,1.0
8,1.0,3.0,27.000000,0.0,2.0,11.1333,0.0
9,1.0,2.0,14.000000,1.0,0.0,30.0708,0.0


2. Drop NaN values


In [8]:
train_mvp = train_mvp.dropna(axis = 0)
train_mvp = train_mvp.reset_index(drop=True)
display(train_mvp)

test_mvp = test_mvp.dropna(axis = 0)
test_mvp = test_mvp.reset_index(drop=True)
display(test_mvp)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
709,0,3,female,39.0,0,5,29.1250
710,0,2,male,27.0,0,0,13.0000
711,1,1,female,19.0,0,0,30.0000
712,1,1,male,26.0,0,0,30.0000


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875
...,...,...,...,...,...,...
326,3,female,3.0,1,1,13.7750
327,1,female,37.0,1,0,90.0000
328,3,female,28.0,0,0,7.7750
329,1,female,39.0,0,0,108.9000


3. Split X and Y

In [103]:
X, Y = train_mvp_c.iloc[:,1:], train_mvp_c.iloc[:, 0]

display(X.head(6))
display(Y.head(6))

,1,2,3,4,5,6
0,3.0,22.000000,1.0,0.0,7.2500,1.0
1,1.0,38.000000,1.0,0.0,71.2833,0.0
2,3.0,26.000000,0.0,0.0,7.9250,0.0
3,1.0,35.000000,1.0,0.0,53.1000,0.0
4,3.0,35.000000,0.0,0.0,8.0500,1.0
5,3.0,29.087022,0.0,0.0,8.4583,1.0


0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
5    0.0
Name: 0, dtype: float64

4. Create Pipeline

In [104]:
pipe = Pipeline([("scaling", StandardScaler()), ("model", KNeighborsClassifier(n_neighbors=5))])
# pipe.get_params()

5. Create GridSearchCV

In [105]:
model = GridSearchCV(estimator = pipe, 
                     param_grid = {'model__n_neighbors': [6,7,8,9,10,11,12,13,14, 18, 22]},
                     cv=10)

6. Model.fit

In [106]:
# %%timeit 

model.fit(X, Y)
df_results = pd.DataFrame(model.cv_results_)

highest_acc = df_results['mean_test_score'].max()
print(highest_acc)
print()
display(df_results)


0.8384269662921348



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007408,0.003030,0.007071,0.002162,6,{'model__n_neighbors': 6},0.755556,0.842697,0.764045,0.876404,0.831461,0.842697,0.865169,0.820225,0.887640,0.797753,0.828365,0.042471,8
1,0.004811,0.000610,0.004896,0.000816,7,{'model__n_neighbors': 7},0.766667,0.831461,0.764045,0.853933,0.831461,0.865169,0.865169,0.853933,0.887640,0.831461,0.835094,0.038794,4
2,0.004691,0.001168,0.005013,0.000764,8,{'model__n_neighbors': 8},0.744444,0.831461,0.764045,0.887640,0.831461,0.842697,0.865169,0.831461,0.865169,0.808989,0.827253,0.042426,11
3,0.004807,0.000610,0.004804,0.000403,9,{'model__n_neighbors': 9},0.744444,0.842697,0.752809,0.887640,0.820225,0.842697,0.865169,0.820225,0.898876,0.808989,0.828377,0.048487,7
4,0.005196,0.000747,0.005406,0.001974,10,{'model__n_neighbors': 10},0.755556,0.842697,0.775281,0.887640,0.865169,0.831461,0.865169,0.831461,0.898876,0.797753,0.835106,0.044745,3
5,0.004987,0.000435,0.004609,0.000671,11,{'model__n_neighbors': 11},0.777778,0.831461,0.775281,0.876404,0.820225,0.820225,0.842697,0.831461,0.887640,0.820225,0.828340,0.034091,9
6,0.004926,0.000534,0.004786,0.000738,12,{'model__n_neighbors': 12},0.800000,0.808989,0.786517,0.887640,0.865169,0.831461,0.842697,0.842697,0.887640,0.831461,0.838427,0.032757,1
7,0.004414,0.000493,0.004777,0.000599,13,{'model__n_neighbors': 13},0.788889,0.797753,0.775281,0.865169,0.842697,0.842697,0.842697,0.831461,0.865169,0.831461,0.828327,0.029424,10
8,0.003802,0.000872,0.004498,0.000677,14,{'model__n_neighbors': 14},0.800000,0.820225,0.786517,0.876404,0.876404,0.853933,0.842697,0.820225,0.887640,0.820225,0.838427,0.032757,1
9,0.003500,0.000495,0.004390,0.000668,18,{'model__n_neighbors': 18},0.800000,0.820225,0.741573,0.876404,0.876404,0.831461,0.831461,0.820225,0.898876,0.831461,0.832809,0.042295,6


In [ ]:
df_preds = pd.DataFrame(model.predict(test_mvp))
display(df_preds.head(50))

Notes from feature engineering:

Null accuracy                    = 61.7%

Base MVP of just numeric columns = 58.6% (train) base

OHC of sex column:               = 64.7% (train) +6%

IterativeImputer                 = 83.8% (train) +19%